In [48]:
import nltk
import random
from nltk.corpus import movie_reviews
from nltk.classify.scikitlearn import SklearnClassifier
import pickle

from sklearn.naive_bayes import MultinomialNB, BernoulliNB
from sklearn.linear_model import LogisticRegression, SGDClassifier
from sklearn.svm import SVC, LinearSVC, NuSVC

from nltk.classify import ClassifierI
from statistics import mode

from nltk.tokenize import word_tokenize

In [49]:
class VoteClassifier(ClassifierI):
    def __init__(self, *classifiers):
        self._classifiers = classifiers

    def classify(self, features):
        votes = []
        for c in self._classifiers:
            v = c.classify(features)
            votes.append(v)
        return mode(votes)

    def confidence(self, features):
        votes = []
        for c in self._classifiers:
            v = c.classify(features)
            votes.append(v)

        choice_votes = votes.count(mode(votes))
        conf = choice_votes / len(votes)
        return conf

In [50]:
short_pos = open("positive.txt","r").read()
short_neg = open("negative.txt","r").read()

documents = []

for r in short_pos.split('\n'):
    documents.append( (r, "pos") )

for r in short_neg.split('\n'):
    documents.append( (r, "neg") )

In [51]:
print(documents[0:2])

[('the rock is destined to be the 21st century\'s new " conan " and that he\'s going to make a splash even greater than arnold schwarzenegger , jean-claud van damme or steven segal . ', 'pos'), ('the gorgeously elaborate continuation of " the lord of the rings " trilogy is so huge that a column of words cannot adequately describe co-writer/director peter jackson\'s expanded vision of j . r . r . tolkien\'s middle-earth . ', 'pos')]


In [52]:
all_words = []

short_pos_words = word_tokenize(short_pos)
short_neg_words = word_tokenize(short_neg)

for w in short_pos_words:
    all_words.append(w.lower())

for w in short_neg_words:
    all_words.append(w.lower())

all_words = nltk.FreqDist(all_words)

word_features = list(all_words.keys())[:5000]

In [54]:
def find_features(document):
    words = word_tokenize(document)
    features = {}
    for w in word_features:
        features[w] = (w in words)

    return features

#print((find_features(movie_reviews.words('neg/cv000_29416.txt'))))

featuresets = [(find_features(rev), category) for (rev, category) in documents]

random.shuffle(featuresets)

# positive data example:      
training_set = featuresets[:10000]
testing_set =  featuresets[10000:]


In [55]:
classifier = nltk.NaiveBayesClassifier.train(training_set)
print("Classifier accuracy percent:", (nltk.classify.accuracy(classifier, testing_set))*100)
classifier.show_most_informative_features(15)

Classifier accuracy percent: 74.84939759036145
Most Informative Features
              engrossing = True              pos : neg    =     21.6 : 1.0
                captures = True              pos : neg    =     17.0 : 1.0
               inventive = True              pos : neg    =     15.7 : 1.0
              refreshing = True              pos : neg    =     13.0 : 1.0
               wonderful = True              pos : neg    =     12.6 : 1.0
                    warm = True              pos : neg    =     12.2 : 1.0
                haunting = True              pos : neg    =     11.7 : 1.0
            refreshingly = True              pos : neg    =     11.7 : 1.0
                  stupid = True              neg : pos    =     11.0 : 1.0
                delicate = True              pos : neg    =     11.0 : 1.0
               realistic = True              pos : neg    =     10.3 : 1.0
                chilling = True              pos : neg    =     10.3 : 1.0
             mesmerizing = 

In [76]:
save_classifier=open("NB.pickle", "wb")
pickle.dump(classifier, save_classifier)
save_classifier.close()

In [77]:
MNB=SklearnClassifier(MultinomialNB())
MNB.train(training_set)
print("MNB accuracy percent:", (nltk.classify.accuracy(MNB, testing_set))*100)

MNB accuracy percent: 73.94578313253012


In [78]:
BNB=SklearnClassifier(BernoulliNB())
BNB.train(training_set)
print("BNB accuracy percent:", (nltk.classify.accuracy(BNB, testing_set))*100)

BNB accuracy percent: 73.94578313253012


In [79]:
LogisticRegression=SklearnClassifier(LogisticRegression())
LogisticRegression.train(training_set)
print("LogisticRegression accuracy percent:", (nltk.classify.accuracy(LogisticRegression, testing_set))*100)

LogisticRegression accuracy percent: 72.89156626506023


In [80]:
SGDClassifier=SklearnClassifier(SGDClassifier())
SGDClassifier.train(training_set)
print("SGDClassifier accuracy percent:", (nltk.classify.accuracy(SGDClassifier, testing_set))*100)

C:\Users\manth\Anaconda3\lib\site-packages\sklearn\linear_model\stochastic_gradient.py:128: FutureWarning: max_iter and tol parameters have been added in <class 'sklearn.linear_model.stochastic_gradient.SGDClassifier'> in 0.19. If both are left unset, they default to max_iter=5 and tol=None. If tol is not None, max_iter defaults to max_iter=1000. From 0.21, default max_iter will be 1000, and default tol will be 1e-3.
  "and default tol will be 1e-3." % type(self), FutureWarning)


SGDClassifier accuracy percent: 70.78313253012048


In [81]:
SVC=SklearnClassifier(SVC())
SVC.train(training_set)
print("SVC accuracy percent:", (nltk.classify.accuracy(SVC, testing_set))*100)

SVC accuracy percent: 49.6987951807229


In [82]:
LinearSVC=SklearnClassifier(LinearSVC())
LinearSVC.train(training_set)
print("LinearSVC accuracy percent:", (nltk.classify.accuracy(LinearSVC, testing_set))*100)

LinearSVC accuracy percent: 68.22289156626506


In [83]:
NuSVC=SklearnClassifier(NuSVC())
NuSVC.train(training_set)
print("NuSVC accuracy percent:", (nltk.classify.accuracy(NuSVC, testing_set))*100)

NuSVC accuracy percent: 74.09638554216868


In [84]:
voted_classifier = VoteClassifier(
                                  NuSVC,
                                  LinearSVC,
                                  MNB,
                                  BNB,
                                  LogisticRegression)

print("voted_classifier accuracy percent:", (nltk.classify.accuracy(voted_classifier, testing_set))*100)

voted_classifier accuracy percent: 74.09638554216868
